In [1]:
from tensorflow import keras
import tensorflow as tf

import numpy as np

from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Reshape, Concatenate
from tensorflow.keras.models import Model

In [2]:
query_category_input = Input(shape=(1,), dtype='int32', name='query_category_input')
user_sex_input = Input(shape=(1,), dtype='int32', name='user_sex_input')
user_age_input = Input(shape=(1,), dtype='int32', name='user_age_input')
user_power_input = Input(shape=(1,), dtype='int32', name='user_power_input')

In [3]:
query_category_layer = Embedding(output_dim=16, input_dim=1000000, input_length=1)(query_category_input)
user_sex_layer = Embedding(output_dim=2, input_dim=2, input_length=1)(user_sex_input)
user_age_layer = Embedding(output_dim=8, input_dim=150, input_length=1)(user_age_input)
user_power_layer = Embedding(output_dim=2, input_dim=7, input_length=1)(user_power_input)

W0703 23:53:25.977761 140735647314816 deprecation.py:506] From /Users/tangfei/.local/share/virtualenvs/notebook-w05_uWGA/lib/python3.7/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
# query_category_layer = Reshape((16,))(query_category_layer)
# user_sex_layer = Reshape((2,))(user_sex_layer)
# user_age_layer = Reshape((8,))(user_age_layer)
# user_power_layer = Reshape((2,))(user_power_layer)

In [5]:
embedding_concat = keras.layers.concatenate([query_category_layer, user_sex_layer, user_age_layer, user_power_layer])

In [6]:
x = LSTM(32)(embedding_concat)

W0703 23:53:26.052407 140735647314816 deprecation.py:506] From /Users/tangfei/.local/share/virtualenvs/notebook-w05_uWGA/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
x = Dense(64, activation='relu', name='Relu_1')(x)
x = Dense(3, activation='relu', name='Relu_2')(x)

In [8]:
sub_model = Model(inputs=[query_category_input, user_sex_input, user_age_input, user_power_input], outputs=[x])

In [9]:
sub_model.predict([[2345], [0], [28], [6]])

array([[0.        , 0.00297655, 0.        ]], dtype=float32)

In [10]:
feature_input = Input(shape=(3,), dtype='float', name='feature_input')

In [11]:
# x = keras.layers.dot([feature_input, x], axes=1)
x = keras.layers.dot([feature_input, sub_model.output], axes=1)

In [12]:
x = Dense(1, activation='sigmoid', name='Sigmoid')(x)

In [13]:
model = Model(inputs=[query_category_input, user_sex_input, user_age_input, user_power_input, feature_input], outputs=[x])

In [14]:
query_category_data = np.random.randint(1000000, size=(1000, 1))
user_sex_data = np.random.randint(2, size=(1000, 1))
user_age_data = np.random.randint(150, size=(1000, 1))
user_power_data = np.random.randint(7, size=(1000, 1))

feature_data = np.random.random(size=(1000, 3))

labels = np.random.random(size=(1000, 1))

In [15]:
model.compile(optimizer='rmsprop', loss='mse')

In [16]:
model.fit([query_category_data, user_sex_data, user_age_data, user_power_data, feature_data], [labels], epochs=10, batch_size=32)

W0703 23:53:26.749093 140735647314816 deprecation.py:323] From /Users/tangfei/.local/share/virtualenvs/notebook-w05_uWGA/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
1000/1000 [==============================] - 4s 4ms/sample - loss: 0.0840
Epoch 2/10
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.0841
Epoch 3/10
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.0837
Epoch 4/10
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.0826
Epoch 5/10
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.0794
Epoch 6/10
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.0733
Epoch 7/10
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.0645
Epoch 8/10
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.0552
Epoch 9/10
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.0477
Epoch 10/10
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.0422


In [17]:
model.predict([[2345], [0], [28], [6], [[20.4, 23.4, 29.8]]])

array([[0.]], dtype=float32)

In [18]:
keras.utils.plot_model(model, show_layer_names=False)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [19]:
sub_model.predict([[2345], [0], [28], [5]])

array([[0.33960158, 0.        , 0.24778432]], dtype=float32)

In [20]:
sub_model.save('ranking_weights_model.h5')

In [21]:
model.predict([[1234], [1], [14], [5], [[1, 1, 1]]])

array([[0.51747835]], dtype=float32)

In [22]:
restore_model = keras.models.load_model('ranking_weights_model.h5')

W0703 23:53:55.555059 140735647314816 deprecation.py:506] From /Users/tangfei/.local/share/virtualenvs/notebook-w05_uWGA/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0703 23:53:55.556041 140735647314816 deprecation.py:506] From /Users/tangfei/.local/share/virtualenvs/notebook-w05_uWGA/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Orthogonal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0703 23:53:55.557030 140735647314816 deprecation.py:506] From /Users/tangfei/.local/share/virtualenvs/notebook-w05_uWGA/li

In [23]:
weights = restore_model.predict([[2345], [0], [28], [5]])

In [24]:
weights.tolist()

[[0.3396015763282776, 0.0, 0.2477843165397644]]

In [25]:
type(restore_model)

tensorflow.python.keras.engine.training.Model

In [26]:
with tf.keras.backend.get_session() as sess:
    tf.saved_model.simple_save(
        sess,
        'ranking_weights_model/1',
        inputs={t.name: t for t in restore_model.input},
        outputs={t.name: t for t in restore_model.outputs})

W0703 23:53:56.311916 140735647314816 deprecation.py:323] From <ipython-input-26-044489ccd386>:6: simple_save (from tensorflow.python.saved_model.simple_save) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
W0703 23:53:56.313003 140735647314816 deprecation.py:323] From /Users/tangfei/.local/share/virtualenvs/notebook-w05_uWGA/lib/python3.7/site-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:201: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
